## I couldn't access de json file from Zalando (as many people in class), that's why I decided to work with the Weather API we didn't have time to work with in class (Alberto suggested to do this otherwise)

In [ ]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
municipios_url = "https://www.el-tiempo.net/api/json/v2/municipios"
resp_mun = requests.get(municipios_url)
mun_dic = resp_mun.json()

In [ ]:
codigos_municipios = pd.json_normalize(mun_dic) #sacamos un data frame con la info de los municipios, para luego poder pasar el código de provincia y de municipio a nuestra clase más abajo. 

In [ ]:
#creamos una clase, con sus clientes

class Tiempo():
    def __init__(self, url = "https://www.el-tiempo.net/api/json/v2/",codigo_provincia = "08", codigo_municipio = "08096"):
        self.resp = requests.get(f'{url}provincias/{codigo_provincia}/municipios/{codigo_municipio}').json()
    def info_municipio(self): #esta función nos da info general de los municipios 
        d = self.resp
        for k, v in d.items():
            if k == 'municipio':
                print(f'La ciudad es {v["NOMBRE"]}, tiene una población de {v["POBLACION_CAPITAL"]} habitantes y pertenece a la provicincia de {v["NOMBRE_PROVINCIA"]}')
            if k == 'temperaturas':
                print(f'La temperatura máxima es {v["max"]} y la mínima {v["min"]}')
            if k == "stateSky":
                print(f'La descripción del día es: {v["description"]}')
        print(f'La temperatura actual es {d["temperatura_actual"]} grados')
        print(f'La fecha actual es: {d["fecha"]}')
        print(f'La humedad es del {d["humedad"]}%')
        print(f'El viento es de {d["viento"]}km/h')
        print(f'La lluvia es del {d["lluvia"]}%')
    
    def temp_media(self): #esta nos da la temperatura media de hoy y mañana para un municipio en concreto 
        d = self.resp
        for k, v in d.items():
            if k == "pronostico":
                for x, y in v.items():
                    if x == "hoy":
                        temperatura_media = sum(list(map(int,(y["temperatura"]))))/len(list(map(int,(y["temperatura"]))))
                        print(f'La temperatura media de hoy es {round(temperatura_media,2)}')
                    elif x == "manana":
                        temperatura_media = sum(list(map(int,(y["temperatura"]))))/len(list(map(int,(y["temperatura"]))))
                        print(f'La temperatura media de mañana es {round(temperatura_media,2)}')
    
    def llovera_mañana(self): #esta nos dice si es probable que llueva mañaba o no 
        d = self.resp
        for k, v in d.items():
            if k == "pronostico":
                for x, y in v.items():
                    if x == "manana":
                        prob_precip = sum(list(map(int,(y["prob_precipitacion"]))))/len(list(map(int,(y["prob_precipitacion"]))))
                        if prob_precip <= 50:
                            print("Probablemente no llueva mañana")
                        else:
                            print("Probablemente lloverá mañana")
                            

In [ ]:
Tiempo().info_municipio() #por defecto he puesto Granollers

In [ ]:
Tiempo(codigo_provincia = "28", codigo_municipio = "28096").info_municipio()

In [ ]:
Tiempo(codigo_provincia = "28", codigo_municipio = "28096").temp_media()

In [ ]:
Tiempo(codigo_provincia = "28", codigo_municipio = "28096").llovera_mañana()